## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre: Juan Pablo
- Apellido: Orciuoli
- Legajo: 1636844

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 13 de Octubre 2022 a las 23h59

In [1]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [3]:
london = pd.read_csv('TP2_airbnb_london.csv')
london.head()

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0,Haringey,Private room,35,1,131,1.71,4,262
1,1,Ealing,Private room,70,2,2,0.07,6,62
2,2,Islington,Private room,45,1,14,0.14,2,364
3,5,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,6,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london" 

In [4]:
np.shape(london)

(51827, 9)

In [5]:
london = london.drop('Unnamed: 0', axis=1)

##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [6]:
london.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51827 entries, 0 to 51826
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood                   51827 non-null  object 
 1   room_type                       51827 non-null  object 
 2   price                           51827 non-null  int64  
 3   minimum_nights                  51827 non-null  int64  
 4   number_of_reviews               51827 non-null  int64  
 5   reviews_per_month               51827 non-null  float64
 6   calculated_host_listings_count  51827 non-null  int64  
 7   availability_365                51827 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 3.2+ MB


#### Vemos que para todos los campos, el total de las filas son no nulas

In [7]:
# Si quisiera ver explicitamente los valores nulos:

london.isnull().sum()

neighbourhood                     0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [8]:
print('neighbourhood:', london.neighbourhood.dtypes)
print('room_type:', london.room_type.dtypes)

neighbourhood: object
room_type: object


##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [9]:
## Genero las dummies para la variable neighbourhood, dentro de la variable neigh_dummies.
neigh_dummies = pd.get_dummies(london.neighbourhood)

Imprimir las dimensiones y primeras lineas de estas dummies

In [10]:
np.shape(neigh_dummies)

(51827, 33)

In [11]:
neigh_dummies.head()

,Barking and Dagenham,Barnet,Bexley,Brent,Bromley,Camden,City of London,Croydon,Ealing,Enfield,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
## Genero las dummies para la variable room_type, dentro de la variable room_dummies.
room_dummies = pd.get_dummies(london.room_type)

Imprimir las dimensiones y primeras lineas de estas dummies

In [13]:
np.shape(room_dummies)

(51827, 3)

In [14]:
room_dummies.head()

,Entire home/apt,Private room,Shared room
0,0,1,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


Joinear las variables dummies al dataset



In [15]:
# Luego las dummies no voy a tener que estandarizarlas
london = london.join(room_dummies).join(neigh_dummies)

In [16]:
# Verifico que se hayan agregado
london.columns

Index(['neighbourhood', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365', 'Entire home/apt',
       'Private room', 'Shared room', 'Barking and Dagenham', 'Barnet',
       'Bexley', 'Brent', 'Bromley', 'Camden', 'City of London', 'Croydon',
       'Ealing', 'Enfield', 'Greenwich', 'Hackney', 'Hammersmith and Fulham',
       'Haringey', 'Harrow', 'Havering', 'Hillingdon', 'Hounslow', 'Islington',
       'Kensington and Chelsea', 'Kingston upon Thames', 'Lambeth', 'Lewisham',
       'Merton', 'Newham', 'Redbridge', 'Richmond upon Thames', 'Southwark',
       'Sutton', 'Tower Hamlets', 'Waltham Forest', 'Wandsworth',
       'Westminster'],
      dtype='object')

In [17]:
# Verifico nueva dimension
np.shape(london)

(51827, 44)

##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [18]:
y = np.array(london[["price"]])
x = london.drop(['price', 'neighbourhood','room_type'], axis=1)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [20]:
# Verifico dimensiones
print('x_train shape is', np.shape(x_train))
print('x_test shape is', np.shape(x_test))
print('y_train shape is', np.shape(y_train))
print('y_test shape is', np.shape(y_test))

x_train shape is (36278, 41)
x_test shape is (15549, 41)
y_train shape is (36278, 1)
y_test shape is (15549, 1)


##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [21]:
# Selecciono columnas a estandarizar
col_names = ['minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']

In [22]:
# Creo una copia del x_train
xtrain_scal = x_train.copy()

In [23]:
# Selecciono columnas para no estandarizar las dummies
features_train = xtrain_scal[col_names]

In [24]:
# Calculo el scaler en base a valores seleccionados
scaler = preprocessing.StandardScaler().fit(features_train.values)

In [25]:
# Estandarizo solamente las columnas que necesito
features_train = scaler.transform(features_train.values)

In [26]:
# Reemplazo las columnas por los valores estandarizados
xtrain_scal[col_names] = features_train

In [27]:
# Verifico que las columnas estandarizadas sean las correctas, y las dummies se mantengan con su forma original
pd.DataFrame(xtrain_scal).head()

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Entire home/apt,Private room,Shared room,Barking and Dagenham,Barnet,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
48141,-0.212018,-0.546379,-0.502453,0.043835,-0.219494,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
33779,-0.212018,0.120996,0.648729,-0.138606,-0.747809,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50231,-0.212018,-0.517362,0.502215,-0.152640,0.308821,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
42426,-0.846115,-0.430314,-0.021050,-0.110538,-0.861020,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36553,-0.846115,0.324110,1.339439,-0.138606,-0.544031,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Verifico dimensiones (que coincida con x_train)
np.shape(xtrain_scal)

(36278, 41)

In [29]:
# Verifico mean=0 and std=1 para las columnas estandarizadas
xtrain_scal[col_names].describe().T

,count,mean,std,min,25%,50%,75%,max
minimum_nights,36278.0,-1.718618e-15,1.000014,-0.846115,-0.846115,-0.212018,0.422078,6.763044
number_of_reviews,36278.0,-4.574383e-16,1.000014,-0.546379,-0.488346,-0.343265,0.062964,14.106852
reviews_per_month,36278.0,-2.439810e-16,1.000014,-0.886181,-0.697806,-0.321055,0.341747,9.962845
calculated_host_listings_count,36278.0,3.020536e-17,1.000014,-0.152640,-0.152640,-0.152640,-0.124572,14.344422
availability_365,36278.0,-3.015885e-16,1.000014,-0.944041,-0.936493,-0.408178,0.935252,1.810745


### Con la misma lógica, estandarizo solamente las columnas necesarias del x_test

In [30]:
# Creo una copia del x_test
xtest_scal = x_test.copy()

In [31]:
# Selecciono columnas para no estandarizar las dummies
features_test = xtest_scal[col_names]

In [32]:
# Estandarizo las columnas que necesito con el scaler ya creado para train
features_test = scaler.transform(features_test.values) 

In [33]:
# Reemplazo las columnas por los valores estandarizados
xtest_scal[col_names] = features_test

In [34]:
# Verifico que las columnas estandarizadas sean las correctas, y las dummies se mantengan con su forma original
pd.DataFrame(xtest_scal).head()

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Entire home/apt,Private room,Shared room,Barking and Dagenham,Barnet,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
18515,-0.212018,0.411159,0.041742,2.583979,-0.883662,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1659,2.958465,-0.430314,-0.635014,-0.152640,-0.944041,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8908,-0.846115,0.730338,-0.062911,-0.152640,-0.083642,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
13240,-0.846115,-0.343265,-0.209425,-0.152640,-0.755357,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7038,1.056175,-0.256216,-0.704782,-0.152640,-0.944041,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# Verifico dimensiones (que coincida con x_test)
np.shape(xtest_scal)

(15549, 41)

In [36]:
# Verifico que mean y std no sean exactamente iguales a 0 y 1, para verificar que no estandaricé con el test set
xtest_scal[col_names].describe().T

,count,mean,std,min,25%,50%,75%,max
minimum_nights,15549.0,0.024876,1.014094,-0.846115,-0.846115,-0.212018,0.422078,6.763044
number_of_reviews,15549.0,-0.004230,0.988364,-0.546379,-0.488346,-0.372281,0.033947,14.977341
reviews_per_month,15549.0,0.005638,1.006123,-0.886181,-0.697806,-0.314078,0.355701,8.790732
calculated_host_listings_count,15549.0,-0.010608,0.937691,-0.152640,-0.152640,-0.152640,-0.124572,14.344422
availability_365,15549.0,-0.003505,0.998490,-0.944041,-0.944041,-0.408178,0.950346,1.810745


##### **Objetivo 9**
Crear un modelo lineal y entrenarlo para nuestro dataset.

Obtener las predicciones y el "mean_squared_error" para el test set.

In [37]:
# Creo el modelo lineal
linear = LinearRegression()

In [38]:
# Entreno el modelo con el xtrain_scal
linear.fit(xtrain_scal,y_train)

LinearRegression()

In [39]:
# Prediccion con xtest_scal
prediction = linear.predict(xtest_scal)

In [41]:
mean_squared_error(prediction,y_test)

1530.8816843292277

##### **Objetivo 10**
Crear un SVM Regressor y un GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [42]:
svreg = SVR()

In [43]:
# 3 valores de C y gamma
param_svreg = {'kernel':('linear', 'rbf'), 'C':[1, 5, 10], 'gamma':[0.1, 0.5 ,1]}

In [49]:
# refit=True y lo hago con 3 cv por el tiempo de ejecución
regressor_svr = GridSearchCV(svreg, param_svreg, cv=3, verbose=1, refit=True)

In [50]:
# Fit con training set
regressor_svr.fit(xtrain_scal, y_train.ravel())

Fitting 3 folds for each of 18 candidates, totalling 54 fits


GridSearchCV(cv=3, estimator=SVR(),
             param_grid={'C': [1, 5, 10], 'gamma': [0.1, 0.5, 1],
                         'kernel': ('linear', 'rbf')},
             verbose=1)

##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set.

In [51]:
## Imprimimos los mejores hiperparametros
regressor_svr.best_params_

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}

In [52]:
## Imprimimos el mejor score
regressor_svr.best_score_

0.5039659036538445

In [53]:
# Prediccion con test set
prediction_2 = regressor_svr.predict(xtest_scal)
prediction_2

array([122.87711107,  89.13786632,  45.27582514, ...,  31.9926478 ,
       102.46105503,  92.47960607])

In [55]:
mean_squared_error(y_test,prediction_2)

1544.7988044221338

##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [56]:
svreg_2 = SVR()

In [67]:
# 3 valores de C y gamma
param_svreg_2 = {'kernel':('linear', 'rbf'), 'C':[20, 50, 80], 'gamma':[0.02, 0.05 ,0.1]}

In [68]:
# refit=True y lo hago con 3 cv por el tiempo de ejecución
regressor_svr_2 = GridSearchCV(svreg_2, param_svreg_2, cv=3, verbose=1, refit=True)

In [69]:
# Fit con training set
regressor_svr_2.fit(xtrain_scal, y_train.ravel())

Fitting 3 folds for each of 18 candidates, totalling 54 fits


GridSearchCV(cv=3, estimator=SVR(),
             param_grid={'C': [20, 50, 80], 'gamma': [0.02, 0.05, 0.1],
                         'kernel': ('linear', 'rbf')},
             verbose=1)

In [70]:
## Imprimimos los mejores hiperparametros
regressor_svr_2.best_params_

{'C': 80, 'gamma': 0.1, 'kernel': 'rbf'}

In [71]:
## Imprimimos el mejor score
regressor_svr_2.best_score_

0.50962136897033

In [72]:
# Prediccion con test set
prediction_3 = regressor_svr_2.predict(xtest_scal)
prediction_3

array([130.01945969,  88.02816649,  43.56942984, ...,  32.56513125,
       104.02220609,  94.31586692])

In [73]:
mean_squared_error(y_test,prediction_3)

1524.3332932219398

### Se obtiene una mínima mejora en las predicciones, pero seguiría iterando con un C mayor (porque la mejor opción de C sigue siendo la mayor de las alternativas evaluadas)

### El modelo lineal y las dos pruebas de SVM tienen resultados similares, al menos con los parámetros propuestos